## 功能介绍

该程序根据 Excel 文件中提供的文件名和超链接，递归地更新指定文件夹及其子文件夹中的文件名。通过 Tkinter 提供的文件选择对话框，用户可以选择文件夹和 Excel 文件，程序会读取 Excel 中的文件名和路径信息，并自动修改文件夹中的文件名。


In [4]:
import os
from urllib.parse import unquote  # 用于解码文件路径中的 URL 编码
from openpyxl import load_workbook  # 用于读取 Excel 文件
from tkinter import Tk, filedialog  # 用于文件选择对话框

def update_filenames_from_excel(folder_path, excel_path):
    """
    根据 Excel 文件中修改后的文件名，递归更新文件夹及子文件夹中的文件名。
    :param folder_path: 文件夹路径
    :param excel_path: Excel 文件路径
    """
    try:
        # 加载 Excel 文件
        wb = load_workbook(excel_path)
        ws = wb.active

        # 遍历 Excel 文件中的文件名和超链接
        for row in ws.iter_rows(min_row=2, max_col=1):  # 假设文件名在第 1 列，从第 2 行开始
            cell = row[0]
            if cell.hyperlink:  # 如果存在超链接
                original_path = cell.hyperlink.target  # 获取原文件路径
                original_name = os.path.basename(original_path)  # 获取原文件名
                new_name = cell.value  # 获取新的文件名

                # 解码路径中的 URL 编码
                original_path = unquote(original_path)

                # 如果文件名发生变化，则更新文件名
                if original_name != new_name:
                    # 递归遍历文件夹及子文件夹
                    for root, dirs, files in os.walk(folder_path):
                        # 构造文件完整路径
                        original_full_path = os.path.join(root, original_path)
                        if os.path.isfile(original_full_path):  # 如果文件存在
                            # 构造新的文件路径
                            new_full_path = os.path.join(root, os.path.dirname(original_path), new_name)

                            # 确保路径的目录存在
                            os.makedirs(os.path.dirname(new_full_path), exist_ok=True)

                            # 执行文件重命名操作
                            try:
                                os.rename(original_full_path, new_full_path)
                                print(f"已重命名: {original_full_path} -> {new_full_path}")
                                break  # 成功重命名后退出循环
                            except FileNotFoundError:
                                print(f"文件未找到，跳过: {original_full_path}")
                            except Exception as e:
                                print(f"重命名失败: {original_full_path} -> {new_full_path}，错误: {e}")
    except Exception as e:
        print(f"读取 Excel 文件或更新文件名时发生错误: {e}")


def select_folder_and_file():
    """
    选择文件夹和 Excel 文件。
    """
    # 创建一个隐藏的 Tkinter 主窗口
    root = Tk()
    root.withdraw()

    # 选择文件夹路径
    folder_path = filedialog.askdirectory(title="选择文件夹")
    if not folder_path:
        print("未选择文件夹，退出程序。")
        return None, None

    # 选择 Excel 文件路径
    excel_file = filedialog.askopenfilename(
        title="选择带有文件链接的 Excel 文件",
        filetypes=[("Excel 文件", "*.xlsx"), ("所有文件", "*.*")]
    )
    if not excel_file:
        print("未选择 Excel 文件，退出程序。")
        return None, None

    return folder_path, excel_file


if __name__ == "__main__":
    # 选择文件夹和 Excel 文件
    folder, excel_file = select_folder_and_file()

    if folder and excel_file:
        update_filenames_from_excel(folder, excel_file)